# GIK2NX LAB 3 ANALYSING SENSOR DATA GROUP 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
import matplotlib.pyplot as plt

### Part One                       
1. ADF-test 
2. data for Auto Arima

In [ ]:
data = pd.read_csv('dataset.csv', index_col='date')
data.head()

In [ ]:
data.describe()

In [ ]:
train_size = int(len(data) * 0.7)
train, test = data.iloc[:train_size], data.iloc[train_size:]

In [ ]:
# Checks stationarity of C02 in the training set
adf_result = adfuller(train['CO2'])
print(adf_result)

In [ ]:
# Finds the best ARIMA model with seasonal settings.
train = train["CO2"]
auto_model = auto_arima(train, 
                        seasonal=True, 
                        trace=True, 
                        error_action='ignore', 
                        suppress_warnings=True)

### Part Two                                                  
1. Use Arima with the Auto Arima results                    
2. Stepwise update, iterate in batches,                      
3. forecast values, and update the model with observed data  
4. Then plot and caculate MAE

In [ ]:
data = pd.read_csv('dataset.csv', index_col='date')

data.index.freq = '1min'
data.index = pd.DatetimeIndex(data.index).to_period('1min')
temp_data = data['CO2']

In [ ]:
# Split the data into train and test sets (70% train, 30% test) per instructions.
split_point = int(.7 * len(temp_data))
train, test = temp_data[:split_point], temp_data[split_point:]

In [ ]:
# Model uses the output of Arima Auto, which is (2, 1, 2). 
model = ARIMA(train, order=(auto_model.order))
fitted_model = model.fit()

In [ ]:
# Stepwise Update and Forecast
# Batches can only be in something divisible with the total number.
# [1, 2, 7, 11, 14, 19, 22, 38, 77, 133, 154, 209, 266, 418, 1463, 2926]
batch_size = 77
forecasts = []
true_values = []

In [ ]:
current_train = train.copy()

In [ ]:
# Iterate in batches
for i in range(0, len(test), batch_size):
    batch = test.iloc[i:i+batch_size]
    print(f'{i} out of 2926')
    
    # Fit the ARIMA model on the current training data
    model = ARIMA(current_train, order=auto_model.order)
    fitted_model = model.fit()

    # Forecast the next 'batch_size' values
    forecast = fitted_model.forecast(steps=batch_size)
    forecasts.extend(forecast)  # Append the forecasted values

    # Update the training data with the actual observed values from the current batch
    current_train = pd.concat([current_train, batch])

    # Store the true values for later evaluation
    true_values.extend(batch)

In [ ]:
# Ensure the indices are in the correct format
train.index = train.index.to_timestamp()
test.index = test.index.to_timestamp()

In [ ]:
forecasts = pd.Series(forecasts, index=test.index)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(forecasts.index, forecasts, label='Stepwise Forecast')
plt.title('CO2 Stepwise Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Temperature Levels')
plt.legend()
plt.show()

In [ ]:
# Scatterplots
cols_to_plot = ['Temperature', 'Humidity', 'Light', 'CO2']
sns.pairplot(data[cols_to_plot])
plt.show()

In [ ]:
# Correlation matrix
corr_matrix = data[cols_to_plot].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Calculate Mean Absolute Error (MAE)
MAE = np.mean(abs(forecasts - true_values))
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))